# INF8775 – Analyse et conception d’algorithmes
# TP3 – Automne 2024

Côté, Gabriel, 2082508

Tourigny, François, 2079718

Note finale:

<u>**Date limite de remise :**</u>  4 décembre 23h59 pour les deux groupes

# Instructions

## Rédaction et remise du rapport

- Ce notebook constitue à la fois le sujet du TP, votre code et votre rapport. Il contient déjà du code pour faciliter vos mesures et l'affichage de vos résultats, ainsi qu'un squelette pour votre rapport.

- Complétez directement le notebook, vous êtes libres de créer des nouvelles cellules de code ou de texte.

- Vous pouvez utiliser des fichiers externes pour stocker des exemplaires et des résultats, mais nous devons être capable de comprendre facilement votre démarche et de la reproduire.

- <u>**IMPORTANT**</u> Remettez le fichier du notebook sur Moodle avec le nom `MATRICULE1_MATRICULE2.ipynb`

- Vous pouvez inclure du code trouvé sur Internet, mais vous devez en mentionner la source, sous peine d'être sanctionnés pour plagiat.

## Mise en situation

Ce travail pratique se répartit sur deux séances de laboratoire et est une occasion de mettre en application les connaissances vues en cours. Vous devrez développer l'algorithme de votre choix pour essayer de résoudre le plus efficacement possible le problème donné. Une partie de la note sera accordée en fonction des résultats que vous obtiendrez par rapport aux autres équipes.

## Description du problème

Le problème qu'on vous demande de résoudre cette fois-ci est un peu plus difficile. Vous êtes responsable de la séparation des voteurs d'un pays en circonscriptions. Un des deux candidats principaux, M. T, vient vous voir et vous demande de lui garantir une victoire (pour un montant non négligeable d'argent). Évidemment, vous refusez, mais le problème est intéressant et vous décidez d'essayez de le résoudre. Si ça vous intéresse, ce problème est ce qu'on appelle du *gerrymandering*.

Vous aurez comme entrée à votre problème une carte du pays (représentée par une matrice carrée $n \times n$) qui contient à chaque position le nombre de voteurs pour candidat X (un chiffre entre 0 et 1000). Votre objectif est de créer $n$ circonscriptions de sorte à ce que candidat X gagne l'élection. Quelques spécifications importantes:

- La variable $n$ représente un côté de la matrice. Il y a donc, $n^2$ villes.
- Chaque position de la matrice représente une ville
- Chaque circonscription doit contenir **$n$ villes**. Une solution reste valide si une circonscription ne contient pas exactement $n$ villes, mais il y a une **pénalité** qui y est associée.
- Les villes d'une circonscription doivent être proche les unes des autres. On aimerait garder cette **distance à, au plus, $n/2$**. Encore une fois, on permet de briser cette contrainte, mais il y aura une pénalité qui y est associée. (**Distance manhattan**)
- Les villes d'une circonscription ne doivent **pas être nécessairement voisines** tant qu'on respecte la distance maximale. Voir l'exemple plus bas.
- Le candidat remporte une circonscription si le nombre de voteurs dans cette circonscription est supérieur à $500n$.
- Le candidat cherche à remporter le plus de circonscriptions qu'il peut.

![alt text](distance_example.png)


## Jeu de données

La classe Problem existe pour simplifier l'interface des différentes fonctions utilitaires. Elle permet de générer des jeux de données avec la méthode `generate_sample` ci-dessous. Elle génère une matrice carrée de taille $n$ contenant des nombres entre $1$ et $1000$. Vous pouvez utilisez des exemplaires aléatoires pour tester votre code. La compétition sera faite sur les mêmes exemplaires de tailles différentes pour toutes les équipes d'un même groupe.

In [2]:
import random
from collections.abc import Iterable

def generate_city() -> int:
    return round(min(1000,max(0,random.normalvariate(450,200))))

class Problem():
    def __init__(self, size: int, num_samples: int = 5) -> None:
        self.size = size
        self.num_samples = num_samples

    def generate_sample(self) -> list[list[int]]:
        """Returns a matrix containing values between 0 and 1000. Each value is the number of voters in a given city"""
        return [[generate_city() for _ in range(self.size)] for _ in range(self.size)]

    def generate_dataset(self) -> Iterable[list[list[int]]]:
        """Returns an iterator over as many samples as are described"""
        return (self.generate_sample() for _ in range(self.num_samples))

# Implantations et expérimentations

Ces fonctions auxiliaires vous sont fournies pour vérifier l'exactitude des vos algorithmes, mesurer leurs performance et afficher vos résultats.

Il est recommandé de prendre le temps de lire et comprendre le code.

Exécutez la cellule ci-dessous pour pouvoir utiliser les fonctions auxiliaires.

In [260]:
import matplotlib.pyplot as plt
import time
from collections.abc import Callable
from math import ceil
import math
from scipy.stats import linregress
import numpy as np
from tqdm import trange

class InvalidSolution(Exception):
    def __init__(self):
        super().__init__("Invalid solution, verify your code.")

class Measure():
    """A wrapper to contain information on taken measures"""
    def __init__(self, size: int, mean: int, score:int) -> None:
        self.size = size
        self.mean_score = score
        self.mean = mean

def score_solution(original: list[list[int]], solution: list[list[tuple[int,int]]]) -> int:
    """Returns the score of the current solution. The score function is a penalty that must be minimized."""
    return votes_score(original, solution) + size_score(solution) + distance_score(solution)

def votes_score(original: list[list[int]], solution: list[list[tuple[int,int]]]) -> int:
    """Calculates the part of the score associated to lost districts. 
    It is 5 times the square of the number of lost districts."""
    lost_districts = 0
    for district in solution:
        sum = 0
        for city in district:
            sum += original[city[0]][city[1]]
        if sum <= 500*len(district):
            lost_districts += 1
    return 5 * lost_districts**2

def size_score(solution: list[list[tuple[int,int]]]) -> int:
    """Calculates the part of the score associated to districts having the wrong size.
    It is the square of the difference between the wanted number of cities and the 
    current number of cities in a given district."""
    n = len(solution)
    size_penality = 0
    for district in solution:
        size_penality += (len(district)-n)**2
    return size_penality

def distance_score(solution: list[list[tuple[int,int]]]) -> int:
    """Calculates the part of the score associated to the distance between cities in a district.
    It is the mean square distance between each city and every other city in its district."""
    distance_score = 0
    n = len(solution)
    for district in solution:
        for i,city in enumerate(district):
            for j in range(i+1, len(district)):
                distance_score += (max(0, distance_manhattan(city, district[j])-ceil(n/2)))**2
    return distance_score/len(solution)

def distance_manhattan(city_a: tuple[int,int], city_b: tuple[int,int]) -> int:
    return abs(city_a[0] - city_b[0]) + abs(city_a[1] - city_b[1])

def is_valid_solution(original: list[list[int]], solution: list[list[tuple[int,int]]]) -> bool:
    """Validates solution"""
    n = len(original)

    if len(solution) != n:
        print(f"The solution does not contain {n} districts.")
        return False

    for district in solution:
        if len(district) < 1:
            print("The solution contains empty districts.")
            return False
        for city in district:
            if len(city)!=2:
                print("Solution must contain 2 coordinates per city.")
                return False
        for coord in city:
            if coord < 0 or coord >=n:
                print(f"City coordinates must below {n} and positive.")
                return False

    coord_set = set()
    for district in solution:
        for city in district:
            coord_set.add(city)
    if len(coord_set) != n*n:
        print(f"Solution contained {len(coord_set)} different cities while there should be {n*n} cities in the solution.")
        return False

    # Solution is valid
    return True

def make_problems(sizes: list[int], num_samples: int = 5) -> list[Problem]:
    """Creates problem instances using given sizes and max_numbers"""
    return [Problem(size,num_samples) for size in sizes]

def measure(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], sample: list[list[int]], time_scale: int = 1000) -> tuple[int,int]:
    """Returns a tuple containing the time as well as the score of the solution, in that order.
    
    Parameters:
        time_scale: Controls the level of precision of the time measurements.

    Raises:
        InvalidSolution: If the procedure returns an invalid solution, raises an exception.
    """
    start: int = time.time() * time_scale
    solution: list[int] = procedure(sample)
    end: int = time.time() * time_scale
    if not is_valid_solution(sample, solution):
        raise InvalidSolution()
    return (round(end - start), score_solution(sample, solution))

def measure_mean(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], prob: Problem, time_scale: int = 1000) -> Measure:
    """Generates multiple samples with the specified parameters and returns a Measure 
    instance representing the result as well as the problem.

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.
    """
    results = [measure(procedure,sample,time_scale) for sample in prob.generate_dataset()]
    mean_time = sum(result[0] for result in results) / prob.num_samples
    mean_score = sum(result[1] for result in results) / prob.num_samples
    return Measure(prob.size, mean_time, mean_score)

def measure_range(procedure: Callable[[list[list[int]]],list[list[tuple[int,int]]]], problems: list[Problem], time_scale: int = 1000) -> list[Measure]:
    """Measures the mean time taken for each problem in the given list.

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.

    Returns:
        A list of Measure instances containing the specifications
        of the problem as well as the mean time and the score.
    """
    return [
        measure_mean(procedure, prob, time_scale)
        for prob in problems
    ]

def display_data_as_table(measures: list[Measure]):
    """Prints a table with the data in the given list of measures"""
    print("{: <12} {: <12} {: <12}".format("Taille", "Temps moyen", "Score moyen"))
    for measure in measures:
        print("{: <12} {: <12} {: <12}".format(measure.size, measure.mean, measure.mean_score))

### The different tests are below, the names are in french to avoid confusion

def test_de_puissance(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de puissance"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    # Log both sets of values
    x = list(data.keys())
    y = list(data.values())

    # Perform the lin regression
    m, b, rvalue, _, _ = linregress(x, y)

    # Estimate the values of y based on the lin regression results
    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2f}x + {b:.2f}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

def test_de_rapport(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de rapport"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    plt.plot(x, y, label='Mesures')
    plt.scatter(x, y, label='Mesures')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

def test_de_constantes(
    data: dict[int,int],
    x_label: str,
    y_label: str = "Temps (ms)",
    title: str = "Test de constantes"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    # Perform linear regression
    m, b, rvalue, _, _ = linregress(x, y)

    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2E}x + {b:.2E}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

# Algorithme

Votre algorithme sera en partie noté en fonction d'une évaluation relative entre les équipes. 4 points seront donnés aux équipes qui se classeront dans le premier quartile lors de notre évaluation sur un ensemble d'exemplaires. Les équipes se trouvant dans le quartile dont les algorithmes ont le moins bien performé recevront 1 point.

**IMPORTANT** Votre algo doit retourner une solution après 3 minutes. Si ce n'est pas le cas, vous serez pénalisé.

In [59]:
class County():
    def __init__(self, row: int, col: int, pos: int, voters: int) -> None:
        self.row = row
        self.col = col
        self.pos = pos
        self.voters = voters
        self.neighbors = []
        self.group = -1

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)

    def get_neighbours(self):
        return self.neighbors
    
    def set_group(self, group: int):
        self.group = group
    
    def get_group(self):
        return self.group
    
    def is_in_group(self):
        return self.group != -1


In [60]:
def get_neighbours_idx(idx, n):
    neighbours = []
    if idx - n > 0:
        neighbours.append(idx-n)
    if idx + n < n**2:
        neighbours.append(idx+n)
    if idx // n == (idx-1) // n:
        neighbours.append(idx-1)
    if idx // n == (idx+1) // n:
        neighbours.append(idx+1)
    return neighbours

def set_counties(sample) -> list[County]:
    n = len(sample)
    counties = [County(i, j, i*n+j, county) for i, line in enumerate(sample) for j, county in enumerate(line)]
    for county in counties:
        for neighbor_idx in get_neighbours_idx(county.pos, n):
            county.add_neighbor(counties[neighbor_idx])
    return counties


In [350]:

def gerrymandering(sample):
    n = len(sample)
    target = 500 * n
    #start_time = time.time()
    #sol = quick_baseline([voters for line in sample for voters in line], target, n)
    #print("--- %s seconds for solution 1 ---" % (time.time() - start_time))
    #start_time = time.time()
    counties = set_counties(sample)
    sol = max_to_target(counties, target, n)
    #print("--- %s seconds for solution 2 ---" % (time.time() - start_time))
    return sol

def quick_baseline(counties_voters: list[int], target: int, n: int):
    solution = []
    for group_id in trange(n):
        new_group = []
        group_neighbors = []
        center_row = 0
        center_col = 0
        number_items = 0
        current_total = 0
        while len(new_group) < n:
            selection = None
            if len(group_neighbors) == 0:
                selection = counties_voters.index(max(counties_voters))
            else:
                adjusted_neighbors = [group_neighbors[i] - distance(i//n, i%n, center_row, center_col) for i in range(len(group_neighbors))]
                if current_total < target:
                    selection = adjusted_neighbors.index(max(adjusted_neighbors))
                    selection = counties_voters.index(group_neighbors[selection])
                else:
                    selection = adjusted_neighbors.index(min(adjusted_neighbors))
                    selection = counties_voters.index(group_neighbors[selection])

            current_total += counties_voters[selection]
            new_group.append((selection//n, selection%n))
            center_row = (center_row * number_items + selection//n) / (number_items + 1)
            center_col = (center_col * number_items + selection%n) / (number_items + 1)
            number_items += 1
            counties_voters[selection] = -1
            
            group_neighbors = [counties_voters[idx] for idx in get_neighbours_idx(selection, n) if counties_voters[idx] >= 0]
            
        solution.append(new_group)
    #drawmap_of_discrits(n, solution)
    return solution

def distance(county_row, county_col, center_row, center_col):
    return 50*math.sqrt((center_row - county_row)**2 + (center_col - county_col)**2)
    
def max_to_target(counties: list[County], target: int, n: int):
    solution = []
    counties_available = counties
    group_neighbors = []
    for group_id in trange(n):
        new_group = []
        number_items = 0
        center_row = 0
        center_col = 0
        group_neighbors = []
        while len(new_group) < n:
            selection = None
            if len(group_neighbors) == 0:
                selection = max(counties_available, key=lambda county: county.voters - distance(county.row, county.col, center_row, center_col))
            elif sum(county.voters for county in new_group) < target:
                selection = max(group_neighbors, key=lambda county: county.voters - distance(county.row, county.col, center_row, center_col))
            else:
                selection = min(group_neighbors, key=lambda county: county.voters - distance(county.row, county.col, center_row, center_col))

            new_group.append(selection)
            center_row = (center_row * number_items + selection.row) / (number_items + 1)
            center_col = (center_col * number_items + selection.col) / (number_items + 1)
            number_items += 1
            selection.set_group(group_id)
            
            group_neighbors.extend([neighbor for neighbor in selection.neighbors if not neighbor.is_in_group() and not neighbor in group_neighbors])
            counties_available.remove(selection)
            if selection in group_neighbors:
                group_neighbors.remove(selection)
            
        solution.append([(county.row, county.col) for county in new_group])
    #drawmap_of_discrits(n, solution)
    return solution
    
    



In [351]:
resultats = measure_range(gerrymandering, make_problems([100], 10))

display_data_as_table(resultats)

100%|██████████| 100/100 [00:01<00:00, 66.00it/s]


Taille       Temps moyen  Score moyen 
100          1522.9       34537.715000000004


In [147]:
def drawmap_of_discrits(n, discrits: list[list[tuple[int,int]]]):
    colors = [[0 for _ in range(n)] for _ in range(n)]
    for i,district in enumerate(discrits):
        for city in district:
            colors[city[0]][city[1]] = i+1

    plt.imshow(colors, cmap='tab20')
    plt.show()

# Analyse asymptotique

Effectuer une analyse asymptotique de votre algorithme.

# Analyse hybride

Effectuer une analyse hybride de votre algorithme.

# Analyse code carbon (2 pts)

Effectuer une anlayse code carbon en sélectionnant différent pays pour l'analyse. Commenter vos résultats.

# Conclusion (6 pts)

Faites une synthèse de vos analyses pour mettre en évidence les qualités et défauts de votre algorithme. Mentionnez les améliorations qui vous ont permis d'atteindre votre algorithme actuel ainsi que des pistes d'améliorations potentielles restantes.

 ## Autres critères (2 pts)
 Qualité du code / 1 pt

Présentation générale / 1 pt

- Concision
- Qualité du français

Pénalité retard
- -2 pt / journée de retard, arrondi vers le haut. Les TPs ne sont plus acceptés après 3 jours.